In [1]:
%matplotlib inline
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy.stats import uniform as sp_randFloat
from sklearn import svm
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from time import time
from tabulate import tabulate
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
import os
from scipy.stats import loguniform
from sklearn.linear_model import LogisticRegression

In [2]:
from scipy.stats import randint as sp_randInt

from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from scipy import sparse

In [5]:
import json
with open('GA_output_ET.json', 'r') as fp:
    feature_list = json.load(fp)

In [6]:
feature_list

{'SYN': ['Src Port',
  'Protocol',
  'Flow Duration',
  'Tot Fwd Pkts',
  'TotLen Bwd Pkts',
  'Fwd Pkt Len Min',
  'Bwd Pkt Len Max',
  'Bwd Pkt Len Min',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Std',
  'Flow Byts/s',
  'Flow IAT Mean',
  'Flow IAT Min',
  'Fwd IAT Tot',
  'Fwd IAT Mean',
  'Fwd IAT Max',
  'Bwd IAT Mean',
  'Bwd IAT Max',
  'Fwd Header Len',
  'Bwd Pkts/s',
  'Pkt Len Min',
  'Pkt Len Max',
  'Pkt Len Mean',
  'SYN Flag Cnt',
  'Subflow Fwd Pkts',
  'Subflow Bwd Pkts',
  'Subflow Bwd Byts',
  'Fwd Act Data Pkts',
  'Active Std',
  'Active Max',
  'Active Min',
  'Idle Std',
  'Idle Max',
  'Idle Min',
  'Label'],
 'HTTP': ['Dst Port',
  'Flow Pkts/s',
  'Flow IAT Mean',
  'Flow IAT Max',
  'Fwd IAT Max',
  'SYN Flag Cnt',
  'Label'],
 'ACK': ['Dst Port',
  'Flow Duration',
  'Tot Fwd Pkts',
  'TotLen Fwd Pkts',
  'TotLen Bwd Pkts',
  'Fwd Pkt Len Max',
  'Bwd Pkt Len Max',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Std',
  'Flow Byts/s',
  'Flow Pkts/s',
  'Flow IAT Mean',
  

In [7]:
file_list={
'SYN': ['../csvs\\dos-synflooding-1-dec.pcap_Flow.csv', '../csvs\\dos-synflooding-6-dec.pcap_Flow.csv'],
'HTTP': ['../csvs\\mirai-httpflooding-4-dec.pcap_Flow.csv', '../csvs\\http-flood -maze.csv'],
'ACK': ['../csvs\\mirai-ackflooding-4-dec.pcap_Flow.csv', '../csvs\\mirai-ackflooding-2-dec.pcap_Flow.csv'],
'UDP': ['../csvs\\mirai-udpflooding-4-dec.pcap_Flow.csv', '../csvs\\mirai-udpflooding-2-dec.pcap_Flow.csv'],
'ARP': ['../csvs\\mitm-arpspoofing-6-dec.pcap_Flow.csv', '../csvs\\mitm-arpspoofing-5-dec.pcap_Flow.csv'],
'SP': ['../csvs\\scan-hostport-3-dec.pcap_Flow.csv', '../csvs\\Edge-Port Scanning attack.pcap_Flow.csv'],
'BF': ['../csvs\\mirai-hostbruteforce-5-dec.pcap_Flow.csv','../csvs\\mirai-hostbruteforce-3-dec.pcap_Flow.csv'],
'OS': ['../csvs\\scan-portos-3-dec.pcap_Flow.csv', '../csvs\\BoT-IoT_OSScan.csv'],
'SCHD': ['../csvs\\scan-hostport-3-dec.pcap_Flow.csv', '../csvs\\scan-hostport-5-dec.pcap_Flow.csv'],
'MHDis': ['../csvs\\mirai-hostbruteforce-3-dec.pcap_Flow.csv', '../csvs\\mirai-hostbruteforce-5-dec.pcap_Flow.csv']}

In [8]:
def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

In [9]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

# RandomizedSearchCV  DT

In [10]:
from scipy.stats import randint
lines=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]

for j in file_list:
    print(j)
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  

    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
       
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
        
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))

    nfolds=10
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': np.linspace(1, 32, 32, endpoint=True).astype(int),
        'min_samples_split': randint(2, 10),
        'max_features': randint(1, X_train.shape[1] + 1)  # Adding +1 to ensure scale is positive
    }

    second=time()
    f1=[]
    clf=DecisionTreeClassifier()
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(50)):
        second=time()
        a,b,clf=run_random_search(DecisionTreeClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(10):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("DT_HPO.csv",index=False)

final_parametres=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))

SYN
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9902731579888235   1.941    24      


100%|██████████| 50/50 [00:57<00:00,  1.15s/it]


HTTP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.392    24      


100%|██████████| 50/50 [00:10<00:00,  4.75it/s]


ACK
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.323    24      


...

SCHD
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.306    24      


100%|██████████| 50/50 [00:09<00:00,  5.27it/s]


MHDis
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9765208654846879   0.879    24      


100%|██████████| 50/50 [00:19<00:00,  2.62it/s]

    criterion      max_depth    max_features    min_samples_split        F1          Std    Time    No  Attack
--  -----------  -----------  --------------  -------------------  --------  -----------  ------  ----  --------
 0  gini                  26              31                    2  0.99421   0             1.274    27  SYN
 1  gini                   1               6                    7  1         0             0.158    46  HTTP
 2  gini                   9              31                    5  1         0             0.287    22  ACK
 3  entropy                2               8                    8  1         0             0.124    23  UDP
 4  entropy               20              31                    4  0.972819  1.11022e-16   0.182    13  ARP
 5  entropy                3              35                    5  1         0             0.432    22  SP
 6  entropy                7              37                    5  0.934988  0.0109587     0.501    24  BF
 7  gini             

# LR

In [11]:
from scipy.stats import loguniform
from sklearn.linear_model import LogisticRegression
lines=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    second=time()

    param_grid = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty': ['none', 'l1', 'l2', 'elasticnet'],  # Initially, allow all
    'C': loguniform(1e-5, 100)
}

    # Ensure compatibility between the solver and the penalty
    for solver in param_grid['solver']:
        if solver == 'newton-cg':
            param_grid['penalty'] = ['none', 'l2']  # Only 'none' and 'l2' are valid for 'newton-cg'
        elif solver == 'liblinear':
            param_grid['penalty'] = ['l1', 'l2']  # 'liblinear' supports both 'l1' and 'l2'
        else:
            param_grid['penalty'] = ['l2', 'none']  # Other solvers support 'l2' or 'none'
     
    second=time()
    f1=[]
    clf=LogisticRegression()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second = time()
        a, b, clf = run_random_search(LogisticRegression(), param_grid, X, y)
        f1 = []
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict = clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict, average="macro"))
        f1_result = sum(f1) / len(f1)
        f1 = np.array(f1)
        stndtd = f1.std()
        temp = list(a.values())
        temp = temp + [f1_result, stndtd, round(time() - second, 3), i, j]
        lines.append(temp)

results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("LR_HPO.csv",index=False)
print (tabulate(results, headers=list(results.columns)))
    
final_parametres=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))

SYN
default                             0.916938364340683    1.347    0       


100%|██████████| 10/10 [17:22<00:00, 104.28s/it]


HTTP
default                             1.0                  0.097    0       


100%|██████████| 10/10 [00:34<00:00,  3.41s/it]


ACK
default                             0.9265678387171424   0.084    0       


...

SCHD
default                             0.947002997002997    0.171    0       


100%|██████████| 10/10 [00:27<00:00,  2.77s/it]


MHDis
default                             0.6428408906224606   0.07     0       


100%|██████████| 10/10 [01:02<00:00,  6.21s/it]

               C  penalty    Solver           F1          Std     Time    No  Attack
--  ------------  ---------  ---------  --------  -----------  -------  ----  --------
 0   0.00218427   l2         newton-cg  0.946673  0             38.013     0  SYN
 1   0.000629179  l2         newton-cg  0.933263  1.11022e-16   57.744     1  SYN
 2  32.2474       l1         liblinear  0.966818  0.0117961    200.331     2  SYN
 3   0.103041     l2         newton-cg  0.946673  0             47.346     3  SYN
 4   0.00288744   l2         lbfgs      0.946673  0             58.956     4  SYN
 5   0.000382968  l2         lbfgs      0.957531  0             22.569     5  SYN
 6  56.6729       l1         liblinear  0.964364  0.00760785   124.055     6  SYN
 7  99.3737       l1         liblinear  0.929628  0.0373117    188.951     7  SYN
 8   5.09004      l1         liblinear  0.913497  0.0325329    249.906     8  SYN
 9   0.000118388  l2         lbfgs      0.903482  1.11022e-16   54.908     9  SYN
10   0.0